In [1]:
import json
import pandas as pd
import numpy as np
import os

os.environ['HF_HOME'] = '/net/tscratch/people/plgkonkie311/cache/'

In [2]:
cd spatial

/net/tscratch/people/plgkonkie311/spatial


In [3]:
path = 'booksy_wroclaw.json'

with open(path, 'r') as file:
    data = json.load(file)


In [4]:
df_booksy = pd.DataFrame(data)

In [5]:
df_booksy.head()

,id,name,address,rating,services
0,0,Barbershop Crazy Razor,"bulwar Ikara 23a/1, 54-130, Wrocław, Fabryczna","4,9","[{'price': '20,00 zł', 'name': 'wosk'}, {'pric..."
1,1,Barbershop Crazy Razor,"Legnicka 59/U14, 54-203, Wrocław, Fabryczna","5,0","[{'price': '499,00 zł', 'name': 'terapia gen f..."
2,2,Klinika Diagnostyki Skóry by Bellissimki,"Legnicka 72, 54-206, Wrocław, Fabryczna","5,0","[{'price': '20,00 zł', 'name': 'konsultacja ko..."
3,3,Instytut Urody Fantastic Body,"UL.LEGNICKA 58, 54-204, Wrocław, Fabryczna","4,9","[{'price': '79,00 zł', 'name': 'strzyżenie chł..."
4,4,Fryzjerek Magnolia,"Wagonowa 2d, 53-609, Wrocław, Fabryczna","5,0","[{'price': '100,00 zł', 'name': 'strzyżenie kl..."


In [6]:
# import json
# from opencage.geocoder import OpenCageGeocode
# from tqdm import tqdm

# # Replace with your OpenCage API key
# API_KEY = "8b20f361ab754e0ca2e82cc12c69aed8"
# geocoder = OpenCageGeocode(API_KEY)

# def geocode_address(row):
#     """Geocodes a single address from a DataFrame row."""
#     address = row['address']
#     result = geocoder.geocode(address)
#     if result:
#         return {
#             'id': row['id'],
#             'address': address,
#             'lat': result[0]['geometry']['lat'],
#             'lng': result[0]['geometry']['lng'],
#             'rating': row['rating'],
#             'services': row['services'],
#         }
#     else:
#         return {
#             'id': row['id'],
#             'address': address,
#             'lat': None,
#             'lng': None,
#             'rating': row['rating'],
#             'services': row['services'],
#         }

# def save_results_to_file(results, filename='geocoding_results.json'):
#     """Saves the geocoding results to a JSON file."""
#     with open(filename, 'w') as file:
#         json.dump(results, file, indent=4)

# def process_geocoding(df, output_file='geocoding_results.json'):
#     """Processes geocoding for all rows in the DataFrame and saves results to a file."""
#     results = []
#     for _, row in tqdm(df.iterrows(), total=len(df)):
#         result = geocode_address(row)
#         results.append(result)
#         save_results_to_file(results, output_file)  # Save results after each row for data integrity
#     return results



In [7]:
# results = process_geocoding(df_booksy)

In [8]:
import json

# Wczytaj GeoJSON
with open("export.geojson", "r", encoding="utf-8") as file:
    geojson_data = json.load(file)

In [9]:

# Funkcja przetwarzająca dane
def preprocess_geojson(geojson):
    objects = []
    for feature in geojson.get("features", []):
        properties = feature.get("properties", {})
        coordinates = feature.get("geometry", {}).get("coordinates", [])

        cleaned_properties = properties.copy()

        del cleaned_properties["@id"]
        rest_data_str = json.dumps(cleaned_properties)

        # Pobierz kluczowe pola
        obj = {
            "name": properties.get("name", "Unknown"),
            "amenity": properties.get("amenity", "Unknown"),
            "address": properties.get("addr:street", "Unknown") + " " + properties.get("addr:housenumber", "") + ", " + properties.get("addr:city", "Unknown"),
            "lattitude": coordinates[1],
            "longitude": coordinates[0],
            "rest_data": rest_data_str
        }
        objects.append(obj)
    return objects

# Przetwórz dane
processed_data = preprocess_geojson(geojson_data)


In [10]:
# to dataframe

df_geojson = pd.DataFrame(processed_data)

In [11]:
df_geojson = df_geojson[df_geojson['amenity'] != 'bench']
df_geojson = df_geojson[df_geojson['amenity'] != 'Unknown']
df_geojson = df_geojson[df_geojson['amenity'] != 'waste_basket']
df_geojson = df_geojson[df_geojson['name'] != 'Unknown']

df_geojson['amenity'].value_counts()


amenity
fast_food         718
restaurant        470
cafe              261
pharmacy          236
bicycle_rental    219
                 ... 
townhall            1
bus_station         1
baby_hatch          1
hospital            1
nursing_home        1
Name: count, Length: 83, dtype: int64

In [12]:
df_cafe = df_geojson[df_geojson['amenity'] == 'cafe']
df_cafe

,name,amenity,address,lattitude,longitude,rest_data
28,Larks,cafe,"Kotlarska 32A, Wrocław",51.111868,17.032191,"{""addr:city"": ""Wroc\u0142aw"", ""addr:housenumbe..."
30,Czekoladziarnia,cafe,"Więzienna 31a, Wrocław",51.111592,17.032107,"{""addr:city"": ""Wroc\u0142aw"", ""addr:housenumbe..."
39,Kalambur,cafe,"Kuźnicza 29a, Wrocław",51.112828,17.034624,"{""addr:city"": ""Wroc\u0142aw"", ""addr:housenumbe..."
46,Sawara,cafe,"Unknown , Unknown",51.083968,17.037038,"{""amenity"": ""cafe"", ""name"": ""Sawara"", ""wheelch..."
290,Mleczarnia,cafe,"Pawła Włodkowica 5, Wrocław",51.108151,17.025399,"{""addr:city"": ""Wroc\u0142aw"", ""addr:housenumbe..."
...,...,...,...,...,...,...
18424,SweetLand Cafe,cafe,"Unknown , Unknown",51.112292,17.032542,"{""amenity"": ""cafe"", ""name"": ""SweetLand Cafe"", ..."
18426,Pi Kawka,cafe,"Unknown , Unknown",51.111640,17.055902,"{""amenity"": ""cafe"", ""cuisine"": ""breakfast"", ""n..."
18427,ArtCaf,cafe,"Unknown , Unknown",51.109999,17.013002,"{""amenity"": ""cafe"", ""name"": ""ArtCaf"", ""start_d..."
18428,Coccoli,cafe,"Unknown , Unknown",51.149092,16.867147,"{""amenity"": ""cafe"", ""cuisine"": ""ice_cream"", ""n..."


In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sdadas/mmlw-roberta-large")

/net/tscratch/people/plgkonkie311/miniconda3/envs/hallu/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-01-26 14:36:12.696154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737898572.827866  408594 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737898572.896008  408594 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-26 14:36:13.183013: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary i

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/30.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.59M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [14]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [15]:
from uuid import uuid4

from langchain_core.documents import Document

documents = []
uuids = []

for _, row in df_geojson.iterrows():
    doc = Document(
        page_content=row['rest_data'],
        metadata={
            'amenity': row['amenity'],
            "name": row['name'],
            "address": row['address'],
            "lattitude": row['lattitude'],
            "longitude": row['longitude'],
        }
    )
    documents.append(doc)
    uuids.append(str(uuid4()))

vector_store.add_documents(documents=documents, ids=uuids)

['ce767da9-75de-48d1-b4a4-a8d885874cd8',
 'a341a803-14d5-4075-9563-76631772fea1',
 '71457261-a067-4c78-a458-a2f03bb66fc9',
 'fb25dc1d-0a0b-44a2-9910-e13e2851f8f8',
 '08537211-0d35-42de-aebe-08f7af921d89',
 '4ced053f-538d-4f63-a033-719e59b8cf15',
 'afff2a63-492e-4d87-b1ce-5630bb680be0',
 'bf0b5127-afe5-45ba-8745-898f3f865e15',
 'feb0a909-6b97-44da-bb91-ded2f8dcbe38',
 '3e21e281-e16b-4e57-a0b7-bb30588aa61d',
 '16e83e25-579e-45a9-bb2f-de30ee4ec186',
 '547c496e-64a0-41b9-8c45-8af5be63d238',
 '7c23449f-813b-46dc-a3df-ad399ed138bc',
 '252eb178-c82c-4695-920a-762c301a7bcd',
 '11a07f0a-116a-4616-964f-1018e2d445c4',
 'f1252df4-7106-427d-955b-b1d7f987a17f',
 '24444416-bd8d-4182-b4b6-89e41c071973',
 'a20efca4-1768-4bed-bc6e-3087443a415d',
 '2621652c-791d-4eac-b411-4d8d5ec36d2e',
 '4e1b07e0-6823-425d-92f0-07aba9eac5c0',
 '3a7d03f7-d38a-4874-b0bc-8e69c2a73334',
 'fbc64d4e-01db-4d5f-b4c5-3aaeab818c3b',
 '99c0bc79-9373-46ef-8fd2-9da82cb854c8',
 '39872949-e872-4050-b2ae-52a2b254d43a',
 '31ec652a-a49a-

In [16]:
vector_store.save_local('all_wroclaw')

In [17]:
results = vector_store.similarity_search(
    "'name': 'Dworzec Wrocław Główny', 'description': 'Wrocław Główny, Dworzec kolejowy'",
    k=5,
    # filter={"amenity": "cafe"}
)
for res in results:
    print(f"* [{res.metadata}] {res.page_content}")

* [{'amenity': 'bicycle_rental', 'name': 'Wrocław Stadion, stacja kolejowa', 'address': 'Unknown , Unknown', 'lattitude': 51.1367247, 'longitude': 16.9414101}] {"amenity": "bicycle_rental", "name": "Wroc\u0142aw Stadion, stacja kolejowa", "network": "Wroc\u0142awski Rower Miejski", "operator": "Nextbike", "ref": "15199"}
* [{'amenity': 'bicycle_rental', 'name': 'Dworzec Główny, południe', 'address': 'Unknown , Unknown', 'lattitude': 51.0971261, 'longitude': 17.0360183}] {"amenity": "bicycle_rental", "bicycle_rental": "dropoff_point", "name": "Dworzec G\u0142\u00f3wny, po\u0142udnie", "network": "Wroc\u0142awski Rower Miejski", "operator": "NextBike", "ref": "15002", "source:location": "survey"}
* [{'amenity': 'bicycle_rental', 'name': 'Mościckiego (stacja kolejowa)', 'address': 'Unknown , Unknown', 'lattitude': 51.0638471, 'longitude': 17.0833664}] {"amenity": "bicycle_rental", "name": "Mo\u015bcickiego (stacja kolejowa)", "network": "Wroc\u0142awski Rower Miejski", "operator": "NextBi

In [18]:
center = results[1].metadata['lattitude'], results[1].metadata['longitude']

def generate_square_filter(center, radius_km=5, amenity=None):
    lat, lon = center
    
    # Approximate degrees per kilometer for latitude and longitude
    lat_delta = radius_km / 110.574  # 1 degree latitude = 110.574 km
    lon_delta = radius_km / (111.320 * np.cos(np.radians(lat)))  # 1 degree longitude = 111.320 km * cos(latitude)

    # Create the bounding box as a filter
    filter_query = {
        "$and": [
            {"lattitude": {"$gte": lat - lat_delta}},  # min latitude
            {"lattitude": {"$lte": lat + lat_delta}},  # max latitude
            {"longitude": {"$gte": lon - lon_delta}},  # min longitude
            {"longitude": {"$lte": lon + lon_delta}},  # max longitude
        ]
    }
    if amenity:
        filter_query["$and"].append({"amenity": amenity})
    return filter_query

generate_square_filter(center, radius_km=5)



{'$and': [{'lattitude': {'$gte': 51.05190751335214}},
  {'lattitude': {'$lte': 51.14234468664785}},
  {'longitude': {'$gte': 16.964496947974933}},
  {'longitude': {'$lte': 17.107539652025064}}]}

In [19]:
results = vector_store.similarity_search(
    "'description': 'Polska kuchnia', 'cuisine': 'polish'",
    k=5,
    filter=generate_square_filter(center, radius_km=3, amenity='restaurant')
)
for res in results:
    print(f"* [{res.metadata}] {res.page_content}")

* [{'amenity': 'restaurant', 'name': 'Pałacyk', 'address': 'Unknown , Unknown', 'lattitude': 51.1026472, 'longitude': 17.0339506}] {"amenity": "restaurant", "cuisine": "polish", "name": "Pa\u0142acyk"}
* [{'amenity': 'restaurant', 'name': 'Centrum Gastronomiczne Margaret', 'address': 'Unknown , Unknown', 'lattitude': 51.08268, 'longitude': 17.074428}] {"amenity": "restaurant", "cuisine": "polish", "name": "Centrum Gastronomiczne Margaret", "website": "https://www.margaretwroclaw.pl/informacje/restauracja"}
* [{'amenity': 'restaurant', 'name': 'Żak', 'address': 'Unknown , Unknown', 'lattitude': 51.1106027, 'longitude': 17.0315414}] {"amenity": "restaurant", "cuisine": "polish", "name": "\u017bak", "outdoor_seating": "yes"}


In [20]:
# visualisation of results on map

import folium

m = folium.Map(location=center, zoom_start=15)

for res in results:
    folium.Marker(
        location=[res.metadata["lattitude"], res.metadata["longitude"]],
        popup=res.metadata["name"],
    ).add_to(m)

folium.Marker(
    location=center,
    popup="Wrocław Główny",
    icon=folium.Icon(color="red"),
).add_to(m)


display(m)


In [21]:
with open('geocoding_results.json', 'r') as file:
    data = json.load(file)

with open('booksy_wroclaw.json', 'r') as file:
    data_w_name = json.load(file)


booksy_df = pd.DataFrame(data)
booksy_df.head()

booksy_df_2 = pd.DataFrame(data_w_name)
booksy_df_2 = booksy_df_2[['id', 'name']]
booksy_df_2.head()


booksy_df = pd.merge(booksy_df, booksy_df_2, on='id', how='left')
booksy_df

,id,address,lat,lng,rating,services,name
0,0,"bulwar Ikara 23a/1, 54-130, Wrocław, Fabryczna",51.100000,17.033300,"4,9","[{'price': '20,00 zł', 'name': 'wosk'}, {'pric...",Barbershop Crazy Razor
1,1,"Legnicka 59/U14, 54-203, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '499,00 zł', 'name': 'terapia gen f...",Barbershop Crazy Razor
2,2,"Legnicka 72, 54-206, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '20,00 zł', 'name': 'konsultacja ko...",Klinika Diagnostyki Skóry by Bellissimki
3,3,"UL.LEGNICKA 58, 54-204, Wrocław, Fabryczna",51.100000,17.033300,"4,9","[{'price': '79,00 zł', 'name': 'strzyżenie chł...",Instytut Urody Fantastic Body
4,4,"Wagonowa 2d, 53-609, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '100,00 zł', 'name': 'strzyżenie kl...",Fryzjerek Magnolia
...,...,...,...,...,...,...,...
2286,2286,"Długa 73, 49-306, Brzeg",50.861981,17.466747,"5,0","[{'price': '150,00 zł', 'name': 'manicure żelo...",Aleksandra Rabiej Beauty Artist
2287,2287,"Sucha 1, 50-086, Wrocław, Krzyki",51.100000,17.033300,"4,9","[{'price': '229,00 zł 206,10 zł', 'name': 'mak...",Imagine Makeup Małgorzata Kowalkowska
2288,2288,"plac Nowy Targ 28, 50-141, Wrocław",51.110081,17.038434,"4,5","[{'price': '299,00 zł', 'name': 'serwis bateri...",INGLOT Wroclavia Wrocław
2289,2289,"Świętokrzyska 38/1B, 50-327, Wrocław, Śródmieście",51.100000,17.033300,"5,0",[],Malowana Lala Salon Urody


In [22]:
# drop nan values
booksy_df = booksy_df.dropna(subset=['lat', 'lng'])
booksy_df

,id,address,lat,lng,rating,services,name
0,0,"bulwar Ikara 23a/1, 54-130, Wrocław, Fabryczna",51.100000,17.033300,"4,9","[{'price': '20,00 zł', 'name': 'wosk'}, {'pric...",Barbershop Crazy Razor
1,1,"Legnicka 59/U14, 54-203, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '499,00 zł', 'name': 'terapia gen f...",Barbershop Crazy Razor
2,2,"Legnicka 72, 54-206, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '20,00 zł', 'name': 'konsultacja ko...",Klinika Diagnostyki Skóry by Bellissimki
3,3,"UL.LEGNICKA 58, 54-204, Wrocław, Fabryczna",51.100000,17.033300,"4,9","[{'price': '79,00 zł', 'name': 'strzyżenie chł...",Instytut Urody Fantastic Body
4,4,"Wagonowa 2d, 53-609, Wrocław, Fabryczna",51.100000,17.033300,"5,0","[{'price': '100,00 zł', 'name': 'strzyżenie kl...",Fryzjerek Magnolia
...,...,...,...,...,...,...,...
2286,2286,"Długa 73, 49-306, Brzeg",50.861981,17.466747,"5,0","[{'price': '150,00 zł', 'name': 'manicure żelo...",Aleksandra Rabiej Beauty Artist
2287,2287,"Sucha 1, 50-086, Wrocław, Krzyki",51.100000,17.033300,"4,9","[{'price': '229,00 zł 206,10 zł', 'name': 'mak...",Imagine Makeup Małgorzata Kowalkowska
2288,2288,"plac Nowy Targ 28, 50-141, Wrocław",51.110081,17.038434,"4,5","[{'price': '299,00 zł', 'name': 'serwis bateri...",INGLOT Wroclavia Wrocław
2289,2289,"Świętokrzyska 38/1B, 50-327, Wrocław, Śródmieście",51.100000,17.033300,"5,0",[],Malowana Lala Salon Urody


In [23]:
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

booksy_vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [24]:
from uuid import uuid4

from langchain_core.documents import Document

documents = []
uuids = []

for _, row in booksy_df.iterrows():
    page_content = 'Name: ' + row['name'] + ' Rating: ' + str(row['rating']) + ' Address: ' + row['address']
    page_content = f'{page_content}  Services: {row["services"]}'

    doc = Document(
        page_content=page_content,
        metadata={
            'name': row['name'],
            'rating': row['rating'],
            'lattitude': float(row['lat']),
            'longitude': float(row['lng']),
            'address': row['address'],
        }
    )

    documents.append(doc)
    uuids.append(str(uuid4()))

booksy_vector_store.add_documents(documents=documents, ids=uuids)

['f2c90f5e-54f0-4016-b59a-471c1b2e0f7f',
 '73e69831-b283-4689-a053-7321388dd22e',
 'bf01b4de-bac7-41a0-8241-20177ec12291',
 '1cc62684-be7f-4635-b2d9-0ed670e4d4be',
 'b5139b15-b804-417a-a241-f43ce666e67a',
 '1c438d57-31e0-46d5-b552-8aaa84ed5703',
 '968a353f-92b2-43b4-9efa-071ed3364dc0',
 '18b0c328-d3dc-42db-a15b-4c3ad838810c',
 'e9bd6aad-8516-43dd-a528-7989f0c85f79',
 '56298fe2-8b03-40ea-b434-148fe5ac16db',
 'baca2e56-82ae-4a89-b7fe-ce1256ed8490',
 'd0c805ba-dcc0-4d68-a113-268af3361468',
 'd26eb423-d684-4ef5-8dfa-5dd4e95bf1e8',
 '3cc2c545-6a6b-4dde-a51a-62d34e9f5661',
 'bde0038b-3dde-4e9b-8b1d-e3a15889e71a',
 '30c554e8-f580-46d5-9494-168f9239e58f',
 '3d5cbf0f-4abb-40f7-831b-6e35ad499a83',
 'd050edcc-804e-43b0-b588-e244f273edf7',
 '19cb4075-ebd8-43ba-8636-73093ccdd2ae',
 '212fc1ad-b3c0-46c0-9cec-bc905c20ed3c',
 '20978bec-4d43-4313-82b9-6f01ac994493',
 'cc84b7e4-5deb-45a4-a4f0-4b507b705285',
 'd0d3b5b8-b05b-4bb7-ae5c-eed989f76d7e',
 'f1f05409-349c-41ad-a8a6-a7721267ba2e',
 '8adf1f78-3bf4-

In [25]:
booksy_vector_store.save_local('booksy_wroclaw')

In [26]:
results = booksy_vector_store.similarity_search(
    "Fryzjer oferujący strzyżenie męskie do 70 zł",
    k=5,
    filter=generate_square_filter(center, radius_km=1)
)

for res in results:
    print(f"* [{res.metadata}] {res.page_content}")

* [{'name': 'Ortogenic', 'rating': '4,9', 'lattitude': 51.1, 'longitude': 17.0333, 'address': 'Przyjaźni 66/12, 12, 53-030, Wrocław, Krzyki'}] Name: Ortogenic Rating: 4,9 Address: Przyjaźni 66/12, 12, 53-030, Wrocław, Krzyki  Services: [{'price': '90,00 zł', 'name': 'strzyżenie męskie włosy krótkie nożyczki maszynka'}, {'price': '100,00 zł', 'name': 'strzyżenie męskie fade nożyczki maszynka'}, {'price': '140,00 zł', 'name': 'strzyżenie damskie średnie włosy'}, {'price': '100,00 zł', 'name': 'strzyżenie męskie fade nożyczki maszynka'}, {'price': '80,00 zł', 'name': 'strzyżenie męskie fade sama maszynka'}, {'price': '90,00 zł', 'name': 'strzyżenie męskie włosy krótkie nożyczki maszynka'}, {'price': '70,00 zł', 'name': 'strzyżenie męskie sama maszynka'}, {'price': '110,00 zł', 'name': 'strzyżenie męskie klasyczne włosy średnie'}, {'price': '130,00 zł', 'name': 'strzyżenie męskie klasyczne włosy długie'}, {'price': '140,00 zł', 'name': 'strzyżenie męskie krótkie włosy odsiwianie'}, {'price

In [27]:
import random

m = folium.Map(location=center, zoom_start=15)

for res in results:
    lat_offset = random.uniform(-0.0005, 0.0005)
    lng_offset = random.uniform(-0.0005, 0.0005)

    lat = res.metadata["lattitude"] + lat_offset
    lng = res.metadata["longitude"] + lng_offset
    
    folium.Marker(
        location=[lat, lng],
        popup=res.metadata["name"],
    ).add_to(m)

folium.Marker(
    location=center,
    popup="Wrocław Główny",
    icon=folium.Icon(color="red"),
).add_to(m)

display(m)